In [6]:
with open('data/songs/寻找独角兽.txt') as file:
    content = file.read().splitlines()
    
file.close()

clean_l = [x for x in content if x != '' and 'https' not in x]

In [257]:
# Split chinese and english
cn_l, en_l = clean_l[::2], clean_l[1::2]

In [8]:
# Split tones
import string

dict_table = str.maketrans('', '', string.digits)
clean_en_l = [x.translate(dict_table) for x in en_l]

In [8]:
from termcolor import colored

red = "\033[31m"
green = "\033[32m"
reset = "\033[39m"

word_book = {}
word_info = []

for s in range(2):
    print(cn_l[s])
    
    i = input()

    correct_cn_w = list(cn_l[s].replace(' ', ''))
    correct_words = clean_en_l[s].split(' ')
    colored_correct_words = correct_words.copy()
    check_words = i.split(' ')

    for w in range(len(check_words)):

        if (check_words[w] != correct_words[w]) and ('/' not in correct_words[w] or check_words[w] not in correct_words[w].split('/')):
            check_words[w] = red + check_words[w] + reset
            colored_correct_words[w] = green + correct_words[w] + reset

            if correct_cn_w[w] in word_book:
                word_book[correct_cn_w[w]][1] += 1
            else:
                word_book[correct_cn_w[w]] = [correct_words[w], 1]

    output_lyrics = ' '.join(check_words)
    correct_lyrics = ' '.join(colored_correct_words)    

    print(correct_lyrics)
    print(output_lyrics)

树上大鸟 与风神在说话
syu soeng daai niu jyu fung san zoi syut waa
sy\
地上巨兽 寄居解语花下
dei soeng geoi sau gei geoi gaai jyu/jyu faa haa



In [13]:
# output json word book to file
import json

with open('data/new words.json', 'w') as f:
    json.dump(word_book, f, indent=4, ensure_ascii=False)

f.close()

In [15]:
import pandas as pd

# load json word book from file
f = open('data/new words.json', 'r')

js_wb = json.loads(f.read())
js_wb

{'上': ['soeng', 2], '与': ['jyu', 1]}

In [61]:
# output as dataframe
data = [[key] + values for key, values in js_wb.items()]
df = pd.DataFrame(data, columns=['Word', 'Pronunciation', 'Counts'])
df

,Word,Pronunciation,Counts
0,上,soeng,2
1,与,jyu,1


In [20]:
# check existing
import os

if not os.path.isfile('data/new words.txt'):
    open('data/new words.txt', 'a').close()
else:
    print('New words book is existing.')

New words book is existing.


In [ ]:
# get list of exisitng songs
import glob

all_f = glob.glob('data/songs/*.txt')
all_songs = [x[11:-4] for x in all_f]
all_songs

In [31]:
title_l1 = "Let's start to learn Cantonese!!!"

game_start = f"+{'-'*(20+len(title_l1))}+\n|{' '*(20+len(title_l1))}|\n|{' '*10}{title_l1}{' '*10}|\n|{' '*(20+len(title_l1))}|\n+{'-'*(20+len(title_l1))}+\n"
print(game_start)

+-----------------------------------------------------+
|                                                     |
|          Let's start to learn Cantonese!!!          |
|                                                     |
+-----------------------------------------------------+



In [42]:
# remove redundent spaces
import re
s = '   this is a text for     finding   multiple spaces'
re.sub('\s+', ' ', s).lstrip()


'this is a text for finding multiple spaces'

In [ ]:
import learnCantonese as lc
W_DIR = 'data/new words.json'

new_w = dict(lc.read_new_words(W_DIR).items())
new_w_list = [f'{i[0]}: {i[1][0]}' for i in new_w.items()]

new_w[new_w_list[0].split(':')[0]].append(1)

new_w

In [36]:
    return '\n\n'.join(['\n'.join(x) for x in zip(lyrics[::2], lyrics[1::2])])


ValueError: Space not allowed in string format specifier

In [28]:
# Tidy up and align lyrics
import learnCantonese as lc
import re

# Split lyrics for better reading
lyric_dir = 'data/songs/南瓜车.txt'
clean_l = list(filter(None, lc.show_lyrics(lyric_dir).split('\n')))
cn_l, en_l = clean_l[::2], clean_l[1::2]
en_l_format = [len(y) for y in (re.sub('\s+', ' ', x).lstrip().split(' ') for x in en_l)]


In [63]:
# Tidy up and align lyrics
import learnCantonese as lc
import re

# Split lyrics for better reading
lyric_dir = 'data/songs/南瓜车.txt'
clean_l = list(filter(None, lc.show_lyrics(lyric_dir).split('\n')))
cn_l, en_l = clean_l[::2], clean_l[1::2]

In [151]:
import pandas as pd
t_en_l = [re.sub('\s+', ' ', x).lstrip().split(' ') for x in en_l]
l = [[len(x) for x in y] for y in t_en_l]
t_cn_l = [list(re.sub('\s+', '', x)) for x in cn_l]

In [283]:
'那'.center(6, chr(12288))

'\u3000\u3000那\u3000\u3000\u3000'

In [296]:
test_c, test_e, test_l = t_cn_l[2], [en_l[2]], l[2]
t_o = ''
for i in range(len(test_c)):
    print(test_c[i], test_l[i], ':', test_c[i].center(test_l[i]))
    t_o += test_c[i].center(test_l[i])

print(t_o + '\n' + test_e[0])

谁 5 :   谁  
知 3 :  知 
十 4 :  十  
二 3 :  二 
点 4 :  点  
刚 5 :   刚  
刚 5 :   刚  
够 4 :  够  
钟 5 :   钟  
融 5 :   融  
洽 4 :  洽  
甜 4 :  甜  
蜜 4 :  蜜  
难 11 :      难     
再 4 :  再  
  谁   知  十   二  点    刚    刚   够    钟    融   洽   甜   蜜       难      再  
seoi4 zi1 sap6 ji6 dim2 gong1 gong1 gau3 zung1 jung4 hap1 tim4 mat6 naan4/naan6 zoi3


In [304]:
len(u'那')

1

In [309]:
test_c, test_e, test_l = t_cn_l[2], [en_l[2]], l[2]
t_o = ''
for i in range(len(test_c)):
    print(test_c[i], test_l[i], ':', ' '*int(test_l[i]/2)+test_c[i]+' '*int(test_l[i]/2))
    t_o += ' '*int(test_l[i]/2)+test_c[i]+' '*int(test_l[i]/2)

print(t_o + '\n' + test_e[0].replace(' ', '  '))

谁 5 :   谁  
知 3 :  知 
十 4 :   十  
二 3 :  二 
点 4 :   点  
刚 5 :   刚  
刚 5 :   刚  
够 4 :   够  
钟 5 :   钟  
融 5 :   融  
洽 4 :   洽  
甜 4 :   甜  
蜜 4 :   蜜  
难 11 :      难     
再 4 :   再  
  谁   知   十   二   点    刚    刚    够    钟    融    洽    甜    蜜       难       再  
seoi4  zi1  sap6  ji6  dim2  gong1  gong1  gau3  zung1  jung4  hap1  tim4  mat6  naan4/naan6  zoi3


In [252]:

o_l = []
for i in range(len(t_cn_l)):
    o_s = ''.join(f'{t_cn_l[i][j].center(l[i][j]-1, chr(12288))}' for j in range(len(t_cn_l[i])))
    o_l.append(o_s)
print('\n\n'.join(['\n'.join(x) for x in zip(o_l, en_l)]))

　那　　天　　专　　车　　驶　　到　　大　　　门　　外　　　整　　车　　装　　　　着　　爱　
naa5 tin1 zyun1 ce1 sai2 dou3 daai6 mun4 ngoi6 zing2 ce1 zong1 zoek6/ oi3

　缘　　　份　　到　　了　　被　　　热　　恋　　　接　　　　载　　　　　出　　　发　　　　向　　　舞　会　　开　
jyun4 fan6 dou3 liu5 bei6/ jit6 lyun2 zip3 zoi2/zoi3 ceot1 faat3 hoeng3 mou5 wui hoi1

　谁　　知　　十　二　　点　　刚　　　刚　　　够　　钟　　　融　　　洽　　甜　　蜜　　　　　难　　　　　　再　
seoi4 zi1 sap6 ji6 dim2 gong1 gong1 gau3 zung1 jung4 hap1 tim4 mat6 naan4/naan6 zoi3

　被　　　你　　赶　　回　　来　　浪　　　漫　　　完　　　被　　　踢　　开　
bei6/ nei5 gon2 wui4 loi4 long6 maan6 jyun4 bei6/ tek3 hoi1

　先　知　　我　　结　　局　已　　惨　　　遭　　篡　　　改　
sin1 zi1 ngo5 git3 guk6 ji5 caam2 zou1 saan3 goi2

　如　　归　　　家　　的　　一　　刻　　再　　没　　南　　　瓜　　车　
jyu4 gwai1 gaa1 dik1 jat1 hak1 zoi3 mut6 naam4 gwaa1 ce1

　就　　靠　　这　　一　　　双　　　腿　　　　　　散　　　　　　步　　返　　　宿　舍　
zau6 kaau3 ze5 jat1 soeng1 teoi2 saan2/saan3 bou6 faan2 suk1 se3

　游　　　　　　　　玩　　　　　　　　过　　童　　　话　　国　　　如　　幻　　　　　　觉　　　　遮　　　　盖　　　　视　野　
jau4 waan2/wun6/waan4 gwo3 tung4 waa6 gwok3 jyu4 waan6 gok3/gaau3 ze1 goi3/koi3 si6 je5

可　　有　　乘　　　　　载　　　　　　着　　　我　　　　　　重　　　　　　回　　现　　实　　班　　车　
ho

In [197]:
'那'.center(9)

'    那    '

In [193]:
l1 = '那'.center(3, chr(12288)) + '天'.center(3, chr(12288)) + '专'.center(4, chr(12288)) + '车'.center(2, chr(12288))
l2 = 'naa5 tin1 zyun1 ce1'
print(l1+'\n'+l2)

　那　　天　　专　　车　
naa5 tin1 zyun1 ce1


In [184]:
print(pd.concat([pd.DataFrame(t_cn_l), pd.DataFrame(t_en_l)]).sort_index())

        0                  1            2            3        4   \
0        那                  天            专            车        驶   
0     naa5               tin1        zyun1          ce1     sai2   
1        缘                  份            到            了        被   
1    jyun4               fan6         dou3         liu5    bei6/   
2        谁                  知            十            二        点   
2    seoi4                zi1         sap6          ji6     dim2   
3    bei6/               nei5         gon2         wui4     loi4   
3        被                  你            赶            回        来   
4        先                  知            我            结        局   
4     sin1                zi1         ngo5         git3     guk6   
5     jyu4              gwai1         gaa1         dik1     jat1   
5        如                  归            家            的        一   
6        就                  靠            这            一        双   
6     zau6              kaau3          ze5      

+-----------------------------------------------------+

         TKINTER EXAMPLES         

+-----------------------------------------------------+


### tkinter

#### Practise

In [3]:
#!/usr/bin/env python3

import tkinter as tk
from tkinter.font import Font

class Pad(tk.Frame):

    def __init__(self, parent, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)

        self.toolbar = tk.Frame(self, bg="#eee")
        self.toolbar.pack(side="top", fill="x")

        self.bold_btn = tk.Button(self.toolbar, text="Bold", command=self.make_bold)
        self.bold_btn.pack(side="left")

        self.clear_btn = tk.Button(self.toolbar, text="Clear", command=self.clear)
        self.clear_btn.pack(side="left")

        # Creates a bold font
        self.bold_font = Font(family="Helvetica", size=14, weight="bold")

        self.text = tk.Text(self)
        self.text.insert("end", "Select part of text and then click 'Bold'...")
        self.text.focus()
        self.text.pack(fill="both", expand=True)

        # configuring a tag called BOLD
        self.text.tag_configure("BOLD", font=self.bold_font)

    def make_bold(self):
        # tk.TclError exception is raised if not text is selected
        try:
            self.text.tag_add("BOLD", "sel.first", "sel.last")        
        except tk.TclError:
            pass

    def clear(self):
        self.text.tag_remove("BOLD",  "1.0", 'end')


def demo():
    root = tk.Tk()
    Pad(root).pack(expand=1, fill="both")
    root.mainloop()


if __name__ == "__main__":
    demo()

: 

: 

In [5]:
from tkinter import *

root = Tk()

txt = Text(root)
txt.pack()

s = ['white', 'white', 'red', 'red', 'white', 'white', 'white', 'white']

txt.tag_config('warning', foreground="red")

for i in s:
    if i == 'white':
        txt.insert('insert', f'{i} ')
    else:
        txt.insert('insert', i, 'warning')
        txt.insert('insert', ' ')

root.mainloop()

In [1]:
from tkinter import *
from tkinter import font

root = Tk()

fontsize = 16

def increase_font():
    global fontsize
    fontsize += 2
    textfont.config(size=fontsize)

textfont = font.Font(family = "consolas" , size = fontsize)
# set the width and height of the Text widget to smallest values
# and let the layout manager to expand it
my_text = Text(root , width=1 , height=1 , font=textfont)
my_text.pack(fill="both", expand=1) # fill the parent frame
my_text.insert(1.0 , "There is a problem that use to test if the text widge is fixed. If the line goes to the next line, it will be correct.")

my_button = Button(root , text="Increase Font" , width=13 , font="arial 11" , command=increase_font)
my_button.pack(pady=5)

mainloop()

: 

: 

In [6]:

import tkinter as tk

root = tk.Tk()

b1 = tk.Button(root, text='b1').pack(side=tk.LEFT)
b2 = tk.Button(root, text='b2').pack(side=tk.LEFT)
b3 = tk.Button(root, text='b3').pack(side=tk.LEFT)
b4 = tk.Button(root, text='b4').pack(side=tk.RIGHT)

root.mainloop()

In [4]:
import os
import json

def read_new_words(dir):
    """ Read new words booklet, if it doesn't exist, create one.
    """
    if not os.path.isfile(dir):
        open(dir, 'a').close()

    with open(dir, 'r') as file:
        try:
            content = json.loads(file.read())
        except json.JSONDecodeError:
            content = {}

    return content

W_DIR = 'data/new words.json'
new_ws = read_new_words(W_DIR)

In [10]:
new_dict = dict(sorted(new_ws.items(), key=lambda item: item[1][1], reverse=True))
for i in new_dict.items():
    print(f'{i[0]}: {i[1][0]}')

NameError: name 'new_ws' is not defined

In [2]:
from tkinter import *

ws = Tk()
ws.title('Learning Cantonese')
ws.geometry('600x400')

var = StringVar()

l = Label(ws, width=4, textvariable=var)
l.pack(fill=X)

list_items = {"Arushi": 22, "Mani": 21, "Haritha": 21}
  
lb = Listbox(ws, selectmode = "single")
for item in list_items.items():
	lb.insert('end', f'{item[0]}: {item[1]}')
      
lb.pack()

def delete_selection(lb):
    global var
    value = lb.curselection()

    lb.delete(value[0])
    if len(value) > 0:
        list_items.pop(list(list_items)[value[0]])

def select_item(lb):
    if lb.curselection() == ():
        var.set('no selection')
    else:
        value = lb.curselection()
        var.set(lb.get(value))

b = Button(ws, text='Delete', command=lambda: delete_selection(lb))
b.pack()

b2 = Button(ws, text='Add selection', command=lambda: select_item(lb))
b2.pack()

ws.mainloop()


In [21]:
from PIL import ImageTk, Image


{'Arushi': 22, 'Haritha': 21}

In [14]:
import tkinter as tk

root = tk.Tk()
root.title('Pack Demo')
root.geometry("350x200")

# box 1
box3 = tk.Label(root, text="Box 3", bg="green", fg="white")
box3.pack(ipadx=20, ipady=20, fill=tk.BOTH, expand=1, anchor=tk.W)


# box 1
box1 = tk.Label(root, text="Box 1", bg="green", fg="white")
box1.pack(ipadx=20, ipady=20, fill=tk.X, expand=1, anchor=tk.W, side=tk.LEFT)

# box 2
box2 = tk.Label(root, text="Box 2", bg="red", fg="white")
box2.pack(ipadx=20, ipady=20, expand=1, anchor=tk.W, side=tk.RIGHT)

# box 1
box4 = tk.Label(root, text="Box 4", bg="green", fg="white")
box4.pack(ipadx=20, ipady=20, fill=tk.BOTH, expand=1, anchor=tk.W)

root.mainloop()

In [6]:
import tkinter as tk
from tkinter import ttk

root = tk.Tk()
root.title('Pack Demo')
root.geometry("300x200")

ipadding = {'ipadx': 10, 'ipady': 10}

# place widgets top down
label1 = tk.Label(root, text='Box 1', bg="red", fg="white")
label1.pack(fill=tk.X)

label2 = tk.Label(root, text='Box 2', bg="green", fg="white")
label2.pack(fill=tk.X)

# place widgets side by side
label4 = tk.Label(root, text='Left', bg="cyan", fg="black")
label4.pack(expand=True, fill=tk.BOTH, side=tk.LEFT)

label5 = tk.Label(root, text='Center', bg="magenta", fg="black")
label5.pack(expand=True, fill=tk.BOTH, side=tk.LEFT)

label6 = tk.Label(root, text='Right', bg="yellow", fg="black")
label6.pack(expand=True, fill=tk.BOTH, side=tk.LEFT)

label3 = tk.Label(root, text='Box 3', bg="blue", fg="white")
label3.pack(fill=tk.X)

root.mainloop()

#### Stimulate pages

In [2]:
# Highlight new words
from tkinter import *
import learnCantonese as lc
W_DIR = 'data/new words copy.json'

new_w = dict(lc.read_new_words(W_DIR).items())
new_w_list = [f'{i[0]}: {i[1][0]}' for i in new_w.items()]

root = Tk()
root.geometry('400x280')

selection = StringVar()

label = Label(root, textvariable=selection)
label.pack(fill=X)

listbox = Listbox(root)
listbox.pack(expand=1, fill="both")

# inserting some items
listbox.insert("end", "A list item")

for item in new_w.items():
    listbox.insert("end", f'{item[0]}: {item[1][0]}')
    if item[1][-1] == 1:
        listbox.itemconfig("end", {'fg': 'light green'})

def add_selection():
    value = listbox.get(listbox.curselection())
    selection.set(value)

def highlight_selection(new_w):
    value = listbox.get(listbox.curselection())
    selection.set(value)
    
    if new_w[value.split(':')[0]][-1] == 0:
        listbox.itemconfig(listbox.curselection()[0], {'fg': 'light green'})
        new_w[value.split(':')[0]][-1] = 1
    else:
        listbox.itemconfig(listbox.curselection()[0], {'fg': 'white'})
        new_w[value.split(':')[0]][-1] = 0

b = Button(root, text='Select', command=add_selection)
b.pack(side=LEFT)

b2 = Button(root, text='highlight', command=lambda: highlight_selection(new_w))
b2.pack(side=LEFT)

root.mainloop()

lc.output_new_words(W_DIR, new_w)

In [11]:
import re

def next_lyrics(display_c_l, cn_l, t, e):
    global N

    N += 1
    value = cn_l[N]
    display_c_l.set(value)

    t.delete("1.0", END)
    e.delete(0, END)
    e.focus_set()

def previous_lyrics(display_c_l, cn_l, t, e):
    global N

    N -= 1
    value = cn_l[N]
    display_c_l.set(value)

    t.delete("1.0", END)
    e.delete(0, END)
    e.focus_set()

def check_lyrics(input_l, cn_l, en_l, w_dir, t, t2):
    global N

    if (len(input_l) == 0) and (t.get('1.0', END) == '\n'):
        t.insert(END, 'There is no input')

    elif t.get('1.0', END) in ['There is no input\n', '\n']:
        t.delete('1.0', END)

        cn_w = list(cn_l.replace(' ', ''))
        en_l = re.sub('\s+',' ', en_l).lstrip().split(' ')
        check_words = re.sub('\s+',' ', input_l).lstrip().split(' ')
        new_w = lc.read_new_words(w_dir)

        for w in range(len(check_words)):
            if (check_words[w] != en_l[w]) and ('/' not in en_l[w] or check_words[w] not in en_l[w].split('/')):
                t.insert(END, en_l[w], 'warning')
                t.insert(END, '  ')

                # add to word book
                lc.add_new_words(new_w, cn_w[w], en_l[w])
                lc.output_new_words(w_dir, new_w)

                if cn_w[w] not in t2.get("1.0", "end"):
                    t2.insert(END, f'{cn_w[w]}: ')
                    t2.insert(END, f'{en_l[w]}\t', 'warning')

            else:
                t.insert(END, f'{en_l[w]}  ')

# Fix duplicate check
from tkinter import *
import learnCantonese as lc

root = Tk()
root.geometry('400x280')

# Get lyrics
lyric_dir = 'data/songs/南昌街王子.txt'
clean_l = lc.read_lyrics(lyric_dir)
cn_l, en_l = clean_l[::2], lc.split_words_tones(clean_l[1::2])
N = 0

display_c_l = StringVar()
display_c_l.set(cn_l[N])

# components to input cantonese
frame = Frame(root)

l = Label(frame, textvariable=display_c_l, bg='#323232', font=("arial", 14), height=2)

e = Entry(frame, bg='black')
e.focus_set()

def clear_entry():
    e.delete(0, 'end')

clear_b = Button(frame, text='x', command=lambda:clear_entry(), width=20, height=20, bg='#616161', fg='white')

# textframe = Frame(root)
t = Text(frame, height=2, font=("arial", 14))
t.tag_config('warning', foreground='#A6FF2E')

# buttons
buttonframe = Frame(root)
b1 = Button(buttonframe, text='Check', command=lambda: check_lyrics(e.get(), cn_l[N], en_l[N], W_DIR, t, t2), bg='#616161', fg='white', width=200)
b2 = Button(buttonframe, text='->', command=lambda: next_lyrics(display_c_l, cn_l, t, e), bg='#616161', fg='white', width=100)
b3 = Button(buttonframe, text='<-', command=lambda: previous_lyrics(display_c_l, cn_l, t, e), bg='#616161', fg='white', width=100)

# words text
frame2 = Frame(root)
t2 = Text(frame2, font=("Helvetica", 14))
t2.tag_config('warning', foreground='#A6FF2E')

# show components on screen
frame.pack()

l.pack(fill='x', padx=10, pady=5)
e.pack(fill='x', padx=10, pady=5)
# clear_b.pack(padx=10, pady=5)
t.pack(fill='x', expand=True, padx=10, pady=5)

buttonframe.pack()
b3.pack(padx=10, pady=5, side=LEFT)
b1.pack(padx=10, pady=5, side=LEFT)
b2.pack(padx=10, pady=5, side=LEFT)

frame2.pack()
t2.pack(fill='x', padx=10, pady=5)

root.bind('<Return>', lambda event: check_lyrics(e.get(), cn_l[N], en_l[N], W_DIR, t, t2))
root.bind('<Right>', lambda event: next_lyrics(display_c_l, cn_l, t, e))
root.bind('<Left>', lambda event: previous_lyrics(display_c_l, cn_l, t, e))

root.mainloop()